In [134]:
#enter name of new ACLED data file, and the start date where you need to start appending from, 'YYYY-MM-DD'
def clean_alced(update_acled_csv, start_date_yyyy_mm_dd_str):

    import pandas as pd

    #read in updated ACLED data
    df_data_to_update = pd.read_csv(update_acled_csv, encoding='latin1')

    #read in current SST excel sheet
    df_data_currently_in_use = pd.read_excel('ACLED_Master_Final.xlsx')

    #edit columns in ACLED data to match SST columns
    df_data_to_update = df_data_to_update[['GWNO', 'EVENT_ID_CNTY', 'EVENT_ID_NO_CNTY', 'EVENT_DATE', 'YEAR',
           'TIME_PRECISION', 'EVENT_TYPE', 'ACTOR1', 'ALLY_ACTOR_1', 'INTER1',
           'ACTOR2', 'ALLY_ACTOR_2', 'INTER2', 'INTERACTION', 'COUNTRY', 'ADMIN1',
           'ADMIN2', 'ADMIN3', 'LOCATION', 'LATITUDE', 'LONGITUDE',
           'GEO_PRECISION', 'SOURCE', 'NOTES', 'FATALITIES']]



    #encode dates in new data set
    df_data_to_update['EVENT_DATE'] = pd.to_datetime(df_data_to_update['EVENT_DATE'], format="%d/%m/%Y", dayfirst=True)

    #create countries filter
    countries = ['Guinea',
     'Uganda',
     'Chad',
     'Ethiopia',
     'South Africa',
     'Niger',
     'Ivory Coast',
     'Togo',
     'Tanzania',
     'Democratic Republic of Congo',

     'Burkina Faso',
     'South Sudan',
     'Mauritania',
     'Sudan',
     'Mali',
     'Liberia',
     'Rwanda',
     'Ghana',
     'Swaziland',
     'Senegal',
     'Gambia',
     'Zambia',
     'Namibia',
     'Angola',
     'Cameroon',
     'Eritrea',
     'Guinea-Bissau',
     'Somalia',
     'Nigeria',
     'Central African Republic',
     'Madagascar',
     'Mozambique',
     'Lesotho',
     'Republic of Congo',
     'Equatorial Guinea',
     'Gabon',
     'Kenya',
     'Benin',
     'Zimbabwe',
     'Burundi',
     'Sierra Leone',
     'Malawi']

    #filter out countries in north africa
    df_ssa = df_data_to_update[df_data_to_update['COUNTRY'].isin(countries)]

    #filter out dates
    df_ssa = df_ssa.set_index('EVENT_DATE').loc[start_date_yyyy_mm_dd_str:].reset_index()

    #creates combined dataframe
    df_final = pd.concat([df_data_currently_in_use, df_ssa])

    #deals with all the characters excel doesn't like
    df_final = df_final.applymap(lambda x: x.encode('unicode_escape').
                 decode('utf-8') if isinstance(x, str) else x)
    
    from openpyxl import load_workbook

    #loads in excel file to be modified
    book = load_workbook('ACLED_Master_Final.xlsx')

    #creates a pandas writer object
    writer = pd.ExcelWriter('ACLED_Master_Final.xlsx', engine='openpyxl')

    writer.book = book

    #creates a dict of sheets
    writer.sheets = dict((ws.title, ws) for ws in book.worksheets)



    df_final.to_excel(writer, 'Main_Sheet', index=False)

    writer.save()